In [19]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import special as sp
from scipy import stats

import random

In [78]:
def dB2Veces(potencia):
    return pow(10, potencia/10)

In [79]:
def dBm2pot(dbm):
    return pow(10, dbm/10)*1e-3

In [2]:
def qfunc(x):
    ## Función Q, calculada con la función de error.
    return 0.5-0.5*sp.erf(x/np.sqrt(2))

In [80]:
def get_SNRPerSimbol(snr, bw, sps):
    ## snr: SNR de la señal en el receptor.
    ## bw: Ancho de banda de la señal.
    ## sps: símbolos por segundo. 
    snr = dB2Veces(snr)
    return snr*bw/sps 

In [81]:
def get_SimbolsPerSecond(bw, gi):
    ## bw: Ancho de banda de la señal.
    ## gi: Guard interval(Evita ISI(Interferencia inter-simbólica) por rebotes de la señal).
    periodo = 1/bw
    new_periodo = 1/bw + gi
    return 1/new_periodo

In [158]:
def get_SER(m, snr, bw, gi):
    ## m: Número de M-QAM
    ## snr: SNR de la señal en el receptor.
    ## bw: Ancho de banda de la señal.
    ## gi: Guard interval(Evita ISI(Interferencia inter-simbólica) por rebotes de la señal).
    sps = get_SimbolsPerSecond(bw, gi)
    gamma = get_SNRPerSimbol(snr, bw, sps)
    #print("Gamma: ", gamma)
    
    return 1-pow((1-(2*(np.sqrt(m)-1)/np.sqrt(m))*qfunc(np.sqrt(3*gamma/(m-1)))), 2)

In [157]:
def get_PER(n, m, snr, bw, gi):
    ## n: N° de bits en un paquete
    ## m: Número de M-QAM
    ## snr: SNR de la señal en el receptor.
    ## bw: Ancho de banda de la señal.
    ## gi: Guard interval(Evita ISI(Interferencia inter-simbólica) por rebotes de la señal).
    ps = get_SER(m, snr, bw, gi)
    ## n_spp: N° de símbolos en un paquete
    n_spp = int(n/np.log2(m))
    #print("SER: ", ps, "\nN° símbolos por paquete: ", n_spp)
    return 1-pow((1-ps), n_spp)

In [84]:
def get_friis_propagation(pt, d, longOnda, ple):
    ## pt: Potencia del transmisor. 
    ## d: Distancia hacia el receptor.
    ## longOnda: Longitud de onda de la portadora.
    ## ple: Path Loss Exponent
    friis = (pt*(longOnda**2)/(((4*np.pi)**2)*((d)**ple)))   
    return 10*np.log10(friis/1e-3)

In [ ]:
def get_exponent_by_enviroment(enviroment):
    if(enviroment == "FREE_SPACE"):
        return 2
    elif( enviroment == "URBAN_AREA_CELLULAR_RADIO"):
        return random.uniform(2.7, 3.5)
    elif( enviroment == "SHADOWED_URBAN_CELLULAR_RADIO"):
        return random.uniform(3.0, 5.0)
    elif( enviroment == "IN_BUILDING_LINE"):
        return random.uniform(1.6, 1.8)
    elif( enviroment == "OBSTRUCTED_IN_BUILDING"):
        return random.uniform(4.0, 6.0)
    elif( enviroment == "OBSTRUCTED_IN_FACTORY"):
        return random.uniform(2.0, 3.0)

In [85]:
pr = get_friis_propagation(1,1000, 0.33, 2)
print(f"Potencia en el receptor: {pr}[dBm]")

Potencia en el receptor: -61.61391848288417[dBm]


In [86]:
PER = get_PER(1e3, 16, 10, 1e6, 8e-6)
print(PER)

Gamma:  90.0
SER:  3.3135471003475914e-05 
N° símbolos por paquete:  250
0.008249787181468404


In [87]:
def get_stdRandNum():
    return np.random.standard_normal()

In [89]:
def isThisPacketWrong(PER):
    area_baja = stats.norm.ppf(PER/2)
    area_alta = stats.norm.ppf(1 - PER/2)
    #print(area_baja, area_alta)
    num = get_stdRandNum()
    if num < area_baja or num > area_alta:
        return True
    else:
        return False

In [90]:
count = 0
max_num = 10000
for i in range(max_num):
    if isThisPacketWrong(PER):
        count +=1
print(f"N° de paquetes con bits errados: {count}\n", f"Probabilidad de PER experimental: {count/max_num}")

N° de paquetes con bits errados: 76
 Probabilidad de PER experimental: 0.0076


In [100]:
def get_RxSNR(S, N):
    S = dB2Veces(S)
    N = dB2Veces(N)
    #print(S, N)
    SNR = S/N
    return 10*np.log10(SNR)

In [159]:
def simulator(max_iter):
    p_tx = 30 # [dBm] Tx
    p_noise = -70 # [dBm] Ruido < --- **Justificar**
    p_inRx = get_friis_propagation(dBm2pot(p_tx), 1000, 0.33, 2)
    #print(dBm2pot(p_tx), p_inRx)
    errs = 0
    count = 0
    while count != max_iter:
        Rx_SNR = get_RxSNR(p_inRx, p_noise)
        #print(Rx_SNR)
        PER = get_PER(1e3, 16, Rx_SNR, 1e6, 8e-6)
        #print(f"PER: {PER}")
        if PER != 0:
             if isThisPacketWrong(PER):
                    errs +=1
        count +=1
    print(f"N° paquetes errados: {errs}, N° total de paquetes {max_iter}, Tasa de trasmisión efectiva: {errs/max_iter}")

In [160]:
simulator(1000)

N° paquetes errados: 151, N° total de paquetes 1000, Tasa de trasmisión efectiva: 0.151
